<a href="https://colab.research.google.com/github/fazeci/QA_with_rag/blob/main/QA_with_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import datetime
import numpy as np
import pandas as pd
import re

# Normal dağılım kullanarak yanıt boyutunu üretme
def generate_normal_distribution_size(mean, std_dev):
    return max(0, int(np.random.normal(mean, std_dev)))

# Rastgele bir tarih oluşturma (son 30 gün içinde)
def generate_random_timestamp():
    end_time = datetime.datetime.now()
    start_time = end_time - datetime.timedelta(days=30)
    random_time = start_time + (end_time - start_time) * random.random()
    return random_time

# Apache log formatına uygun sahte veriler oluşturma
def generate_fake_apache_log_entry():
    ip = ".".join(map(str, random.sample(range(1, 256), 4)))
    timestamp = generate_random_timestamp()
    method = random.choice(['GET', 'POST', 'PUT', 'DELETE', 'HEAD', 'OPTIONS', 'PATCH', 'CONNECT', 'TRACE'])
    uri = random.choice(['/index.html', '/about.html', '/contact.html', '/login', '/products', '/api/data', '/admin'])
    protocol = "HTTP/1.1"
    status = random.choice([200, 201, 204, 301, 302, 304, 400, 401, 403, 404, 500, 502, 503, 504])
    size = generate_normal_distribution_size(mean=1500, std_dev=500)

    log_entry = f'{ip} - - [{timestamp.strftime("%d/%b/%Y:%H:%M:%S +0000")}] "{method} {uri} {protocol}" {status} {size}'
    return log_entry

# Belirli bir sayıda sahte log verisi oluşturma ve dosyaya yazma
def generate_and_save_fake_logs(num_entries, filename='fake_apache_logs.txt'):
    fake_logs = [generate_fake_apache_log_entry() for _ in range(num_entries)]

    with open(filename, 'w') as file:
        for entry in fake_logs:
            file.write(entry + '\n')

# Log dosyasını oku ve ilk 10 log verisini görüntüleme
def display_logs(filename, num_entries=10):
    with open(filename, 'r') as file:
        logs_to_display = [next(file) for _ in range(num_entries)]
        print("".join(logs_to_display))

# Apache/Nginx log formatını regex ile ayıklama
def extract_logs_to_dataframe(log_file_path):
    log_pattern = r'(?P<ip>\d+\.\d+\.\d+\.\d+)\s+\S+\s+\S+\s+\[(?P<timestamp>.*?)\]\s+"(?P<method>\S+)\s+(?P<uri>\S+)\s+(?P<protocol>\S+)"\s+(?P<status>\d+)\s+(?P<size>\d+)'

    # Logları DataFrame'e yükle
    log_df = pd.read_csv(log_file_path, sep='|', names=['log_entry'])
    log_df = log_df['log_entry'].str.extract(log_pattern)

    # Veri türlerini dönüştürme
    log_df['timestamp'] = pd.to_datetime(log_df['timestamp'], format='%d/%b/%Y:%H:%M:%S +0000')
    log_df['status'] = log_df['status'].astype(int)
    log_df['size'] = log_df['size'].astype(int)

    return log_df

# Geçersiz (olumsuz) status kodları veya size değerlerini kontrol etme
def validate_logs(log_df):
    invalid_status = log_df[~log_df['status'].between(100, 599)]  # HTTP status kodları 100-599 aralığındadır
    invalid_size = log_df[log_df['size'] < 0]  # Boyutun negatif olmaması gerekir

    return invalid_status, invalid_size

# Kullanım:

# 1. Sahte logları oluştur ve kaydet
generate_and_save_fake_logs(1000, 'fake_apache_logs.txt')

# 2. Oluşturulan log dosyasının ilk 10 satırını görüntüle
display_logs('fake_apache_logs.txt', num_entries=10)

# 3. Log dosyasını oku ve DataFrame'e yükle
log_df = extract_logs_to_dataframe('fake_apache_logs.txt')

# 4. Geçersiz logları kontrol et
invalid_status, invalid_size = validate_logs(log_df)

# Geçersiz logları görüntüleme
if not invalid_status.empty:
    print("Geçersiz Status Kodları:")
    print(invalid_status)
if not invalid_size.empty:
    print("Geçersiz Boyut Değerleri:")
    print(invalid_size)
else:
    print("Tüm loglar geçerli.")


238.7.9.150 - - [05/Aug/2024:08:43:35 +0000] "HEAD /products HTTP/1.1" 502 689
72.237.200.21 - - [27/Jul/2024:13:53:05 +0000] "GET /api/data HTTP/1.1" 400 1528
191.6.104.99 - - [16/Aug/2024:22:21:32 +0000] "PUT /contact.html HTTP/1.1" 301 1610
46.87.250.208 - - [06/Aug/2024:17:58:11 +0000] "HEAD /products HTTP/1.1" 503 1146
127.97.2.79 - - [15/Aug/2024:11:08:30 +0000] "CONNECT /admin HTTP/1.1" 301 632
251.228.213.163 - - [21/Jul/2024:22:49:50 +0000] "CONNECT /products HTTP/1.1" 403 1884
243.41.174.93 - - [07/Aug/2024:23:55:38 +0000] "HEAD /admin HTTP/1.1" 200 1519
195.41.80.11 - - [28/Jul/2024:07:57:32 +0000] "DELETE /contact.html HTTP/1.1" 200 671
203.10.83.51 - - [17/Aug/2024:18:39:28 +0000] "POST /about.html HTTP/1.1" 200 1424
123.249.2.167 - - [12/Aug/2024:03:37:05 +0000] "DELETE /about.html HTTP/1.1" 503 1454

Tüm loglar geçerli.


In [ ]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip install faiss-cpu
import faiss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 15.8 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import BartTokenizer, BartForConditionalGeneration
from sklearn.cluster import KMeans
import faiss

# Model ve tokenizer yükleme
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [ ]:

# FAISS index oluşturma ve logları ekleme

def setup_faiss_index(log_df):
    log_vectors = sentence_model.encode(log_df['log_text'].tolist(), show_progress_bar=True)
    d = log_vectors.shape[1]
    quantizer = faiss.IndexFlatL2(d)
    index = faiss.IndexIVFFlat(quantizer, d, 100)
    index.train(np.array(log_vectors, dtype='float32'))
    index.add(np.array(log_vectors, dtype='float32'))
    return index

# KMeans kümeleme kullanarak logları gruplama
def cluster_logs(log_vectors, num_clusters=5):
    kmeans = KMeans(n_clusters=num_clusters, n_init=10)
    kmeans.fit(log_vectors)
    return kmeans

# Kullanıcı sorgusunu vektörize etme ve en uygun logları bulma
def find_relevant_logs(query, model, log_df, index):
    query_vector = model.encode([query])
    distances, indices = index.search(np.array(query_vector, dtype='float32'), 5)
    relevant_logs = log_df.iloc[indices[0]]
    return relevant_logs

# Yanıt oluşturma
def generate_final_response(query, selected_logs):
    context = "\n".join(selected_logs['full_log'].tolist())
    input_text = f"Using the logs below, answer the following query: {query}\nLogs: {context}"
    inputs = bart_tokenizer(input_text, return_tensors='pt', max_length=1024, truncation=True)
    outputs = bart_model.generate(inputs['input_ids'], max_length=100, num_beams=5, early_stopping=True)
    return bart_tokenizer.decode(outputs[0], skip_special_tokens=True)

# RAG sistemi ile sorguya yanıt üretme
def rag_system(query, log_df, index):
    relevant_logs = find_relevant_logs(query, sentence_model, log_df, index)
    final_answer = generate_final_response(query, relevant_logs)
    return final_answer, relevant_logs

# Ana döngü
def main():
    # Sahte logları oluştur ve işle
    generate_and_save_fake_logs(1000, 'fake_apache_logs.txt')
    log_df = extract_logs_to_dataframe('fake_apache_logs.txt')
    log_df['full_log'] = log_df.apply(lambda row: f"{row['ip']} {row['timestamp']} {row['method']} {row['uri']} {row['protocol']} {row['status']} {row['size']}", axis=1)
    log_df['log_text'] = log_df.apply(lambda row: f"{row['method']} {row['uri']} {row['status']}", axis=1)

    # FAISS indexi kur
    index = setup_faiss_index(log_df)

    while True:
        query = input("Sorgunuzu girin ('q' ile çıkabilirsiniz): ")
        if query.lower() == 'q':
            break

        answer, logs = rag_system(query, log_df, index)
        print("\nSorgu:", query)
        print("İlgili Loglar:\n", logs.to_string(index=False))
        print("Yanıt:", answer)

if __name__ == "__main__":
    main()


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Sorgunuzu girin ('q' ile çıkabilirsiniz): when was the last time code 200 was received?

Sorgu: when was the last time code 200 was received?
İlgili Loglar:
             ip           timestamp method       uri protocol  status  size                                                             full_log            log_text
 188.26.183.12 2024-07-20 05:00:40  PATCH /products HTTP/1.1     200   597   188.26.183.12 2024-07-20 05:00:40 PATCH /products HTTP/1.1 200 597 PATCH /products 200
170.215.74.146 2024-08-04 13:17:50  PATCH /products HTTP/1.1     200  1344 170.215.74.146 2024-08-04 13:17:50 PATCH /products HTTP/1.1 200 1344 PATCH /products 200
74.162.206.205 2024-07-29 09:03:02  PATCH /products HTTP/1.1     200   607  74.162.206.205 2024-07-29 09:03:02 PATCH /products HTTP/1.1 200 607 PATCH /products 200
196.253.35.129 2024-07-28 02:22:36  PATCH /products HTTP/1.1     201  1400 196.253.35.129 2024-07-28 02:22:36 PATCH /products HTTP/1.1 201 1400 PATCH /products 201
167.162.93.228 2024-08

In [ ]:
#when was the last time code 200 was received?
#What was the most common URI in the last month?
#What was the most common HTTP method used in the last week?
#What is the status of the latest log entry?
#What was the most common error code in the last week?
#Which method has the highest average request size?
#What is the average response time for each HTTP status code?
#what people do most on this site
#Which HTTP method was used most frequently in the last week?
#What was the most common error code in the logs for the past month?
#When was the last time a 500 error code occurred?
#Which URI received the most requests in the last 24 hours?


In [ ]:

# Temel analiz
print(log_df.head())
print(log_df.info())
print(log_df.dtypes)
print(f"Geçersiz status kayıtları: {len(invalid_status)}")
print(f"Geçersiz size kayıtları: {len(invalid_size)}")



               ip                   timestamp   method            uri  \
0  109.106.39.234  27/Jul/2024:13:18:46 +0000   DELETE  /contact.html   
1   47.180.158.23  06/Aug/2024:01:41:02 +0000    TRACE    /index.html   
2    53.40.65.108  27/Jul/2024:15:42:49 +0000     POST  /contact.html   
3  196.100.27.114  28/Jul/2024:14:56:14 +0000     HEAD         /login   
4  155.249.171.66  15/Aug/2024:03:55:54 +0000  CONNECT    /index.html   

   protocol status  size  
0  HTTP/1.1    403  1510  
1  HTTP/1.1    503  1105  
2  HTTP/1.1    404  1146  
3  HTTP/1.1    201  1672  
4  HTTP/1.1    302  1188  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ip         1000 non-null   object
 1   timestamp  1000 non-null   object
 2   method     1000 non-null   object
 3   uri        1000 non-null   object
 4   protocol   1000 non-null   object
 5   status     1000 

In [ ]:
# En sık kullanılanlarıanaliz etme
counts = log_df['status'].value_counts()
print(counts)


status
503    79
200    79
400    77
502    77
404    75
403    74
500    73
301    71
504    71
201    69
304    69
302    66
401    63
204    57
Name: count, dtype: int64


**Proje Raporu: Log Verileri Üzerinde RAG Modeli ile Sorgu Yanıtlama Sistemi**

***1. Giriş***

*Projenin Amacı:*
Bu proje, web trafik log verilerini kullanarak kullanıcı sorgularına uygun yanıtlar veren bir Retrieval-Augmented Generation (RAG) sistemi geliştirmeyi amaçlamaktadır. Sistem, log verileri üzerinde arama yaparak en uygun kayıtları bulmakta ve bu kayıtları kullanarak jeneratif bir dil modeli ile kullanıcıya yanıt oluşturmaktadır.

*Kullanılan Veri Seti:*
- Apache log verileri
- 100,000 satırlık örnek log kayıtları (IP adresleri, erişilen sayfalar, zaman damgaları vb.)

***2. Metodoloji***

*Aşama 1: Veri Hazırlığı ve Ön İşleme*
- Veri Analizi: Log verileri incelendi ve gerekli sütunlar (IP adresi, URI, HTTP metodu, durum kodu vb.) ayıklandı.
- Veri Temizleme: Eksik veya hatalı veriler temizlendi ve sütunlar uygun veri türlerine dönüştürüldü.
- Vektörleştirme: Log kayıtları, dil modeli kullanılarak vektörlere dönüştürüldü. FAISS kütüphanesi ile bu vektörler sorgulanabilir bir veri tabanına yüklendi.

*Aşama 2: RAG Modelinin Kurulumu*
- Bilgi Alma Modülü: FAISS kütüphanesi kullanılarak, kullanıcıdan gelen sorguya en benzer log kayıtlarını bulmak üzere bir bilgi alma modülü oluşturuldu.
- Jeneratif Modül: 'bert-large-uncased-whole-word-masking-finetuned-squad' modeli kullanarak, bulunan log kayıtları üzerinde anlamlı yanıtlar oluşturmak için bir jeneratif modül geliştirildi.
- Sistem Entegrasyonu: Bilgi alma ve jeneratif modülleri entegre edilerek, tam işlevsel bir RAG modeli oluşturuldu.

*Aşama 3: Sistem Entegrasyonu ve Test*
- Sistemin Test Edilmesi: Çeşitli kullanıcı sorguları ile sistem test edildi. Yanıtların doğruluğu ve kalitesi değerlendirildi.
- Sistem Performansı: Modelin performansı, sorgu süresi ve yanıt kalitesi açısından ölçüldü.

***3. Karşılaşılan Zorluklar ve Çözümleri***

- Zorluk: Log verilerinin karmaşıklığı ve tekrarlayan bilgiler.
  
  Çözüm:Veriyi vektörleştirme sürecinde, benzer log kayıtlarını daha iyi işleyebilmek için veri temizliği ve özetleme teknikleri uygulandı.
  
- Zorluk: Modelin tekrarlayan veya anlamsız yanıtlar üretmesi.
  
  Çözüm: Modelde `no_repeat_ngram_size` ve `max_new_tokens` parametreleri ayarlandı. Ayrıca, giriş kontekstinin daha anlamlı hale getirilmesi sağlandı.

- Zorluk: Veri Türleri ile İlgili Sorunlar
  
  Çözüm: Veri türlerini uygun hale getirmek için öncelikle her bir sütunun içeriği analiz edildi ve uygun veri türlerine dönüştürüldü.
  
Zaman Damgaları: timestamp sütunu, datetime formatına dönüştürüldü, böylece zaman bazlı gruplama ve analizler yapılabildi.

Durum Kodları ve Yanıt Boyutları: status ve size sütunları, int64 veri türüne dönüştürüldü, böylece sayısal işlemler ve karşılaştırmalar yapılabildi.

IP Adresleri: IP adresleri, string formatında bırakıldı, ancak gerektiğinde IP’lerin sınıflandırılması ve gruplandırılması için düzenli ifadeler (regex) kullanıldı.

***4. Performans Değerlendirmesi***

-Doğruluk: Sistem, kullanıcı sorgularına yanıt verirken doğru ve alakalı log kayıtlarını kullandı. Yanıtlar, log kayıtlarının genel paternlerine uygun şekilde oluşturuldu.
-Yanıt Süresi: Sistem, büyük veri setiyle çalışmasına rağmen makul sürelerde yanıt üretebildi. FAISS vektör veri tabanı, yüksek verimli arama sağladı.


***5. Sonuç ve Öneriler***

* Genel Değerlendirme: Geliştirilen sistem, log verileri üzerinde başarılı bir
şekilde sorgu yapabilen ve yanıtlar üretebilen bir RAG modeli olarak çalıştı.

* Öneriler: Daha farklı veri türleriyle sistemin test edilmesi, sistemin genelleme yeteneğini artırabilir.

* BART, roberta-base ve T5 dil modelleri denendi ancak yanıtların doğruluğunun artırılması için modelin daha çok geliştirilmesi gerekmektedir.

***6. Kaynaklar***

- Kullanılan kütüphaneler: `transformers`, `sentence-transformers`, `faiss`, `pandas`, `numpy`



FATMA ZEHRA ÇINAR
